# Import Libraris

In [140]:
import mlflow
from mlflow import catboost
!pip install evidently

In [141]:
pip install ydata_profiling

Note: you may need to restart the kernel to use updated packages.


In [143]:
pip install facets-overview

Note: you may need to restart the kernel to use updated packages.


In [144]:
import pandas as pd

from IPython.core.display import HTML, display_html
import base64
from facets_overview.generic_feature_statistics_generator import GenericFeatureStatisticsGenerator

from ydata_profiling import ProfileReport

from evidently import Dataset, DataDefinition, Report
from evidently.presets import DataDriftPreset, DataSummaryPreset

import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.offline as py

# Load data and EDA

In [159]:
df = pd.read_csv('Hotel Reservations.csv')

In [160]:
profile = ProfileReport(df, title="Hotel Bookings Profiling", minimal=True)

profile.to_file("report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 19/19 [00:01<00:00, 12.44it/s][A


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  object 
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   type_of_meal_plan                     36275 non-null  object 
 6   required_car_parking_space            36275 non-null  int64  
 7   room_type_reserved                    36275 non-null  object 
 8   lead_time                             36275 non-null  int64  
 9   arrival_year                          36275 non-null  int64  
 10  arrival_month                         36275 non-null  int64  
 11  arrival_date   

In [ ]:
proto = GenericFeatureStatisticsGenerator().ProtoFromDataFrames([{'name': 'train', 'table': df}])

protostr = base64.b64encode(proto.SerializeToString()).decode("utf-8")
HTML_TEMPLATE = """
<script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
<link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/master/facets-dist/facets-jupyter.html">
<facets-overview id="elem"></facets-overview>
<script>
document.querySelector("#elem").protoInput = "{protostr}";
</script>"""
html = HTML_TEMPLATE.format(protostr=protostr)
display_html(html)

In [ ]:
evidently_report = Report(metrics=[
    DataSummaryPreset(),
], include_tests=True)

run = evidently_report.run(current_data=df, reference_data=None)

In [ ]:
run

# Feature Genering and prepare data

In [161]:
del df['Booking_ID']

In [162]:
def target_to_int(df):
    df['booking_status'] = (df['booking_status'] == 'Canceled').astype('int64')
    return df

In [163]:
df['date'] = df.apply(
    lambda row: ''.join(map(str, [row['arrival_year'], row['arrival_month'], row['arrival_date']])),
    axis=1
)

In [164]:
df = df.sort_values(by='date')
del df['date']

In [165]:
df = target_to_int(df)

In [166]:
df

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
21771,2,0,2,3,Meal Plan 1,0,Room_Type 1,6,2017,10,1,Offline,0,0,0,58.40,0,0
3866,1,0,0,1,Meal Plan 1,0,Room_Type 1,1,2017,10,1,Offline,0,0,0,96.00,0,0
34227,2,0,0,2,Meal Plan 1,0,Room_Type 1,16,2017,10,1,Online,0,0,0,117.00,2,0
34104,1,0,0,2,Meal Plan 1,0,Room_Type 1,17,2017,10,1,Online,0,0,0,73.88,3,0
6144,2,0,1,3,Meal Plan 1,0,Room_Type 1,16,2017,10,1,Online,0,0,0,111.75,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13628,2,0,1,2,Meal Plan 1,0,Room_Type 1,198,2018,9,9,Offline,0,0,0,90.00,0,1
15109,2,0,1,2,Meal Plan 1,0,Room_Type 1,198,2018,9,9,Offline,0,0,0,90.00,0,1
31588,2,0,0,2,Meal Plan 1,0,Room_Type 1,169,2018,9,9,Offline,0,0,0,80.75,0,0
16462,2,0,0,2,Meal Plan 1,0,Room_Type 4,65,2018,9,9,Online,0,0,0,149.40,1,1


In [34]:
fig = go.Figure([go.Bar(x=['no', 'yes'], y=df['booking_status'].value_counts())])
fig.show()

# Calculate historical data

In [ ]:
T = 10000
his_df = df[:T]

In [ ]:
P_his = his_df['booking_status'].sum() / T
P_his

In [ ]:
Avg_rev_his = his_df['avg_price_per_room'].sum() / T
Avg_rev_his

In [ ]:
Loss = P_his * T * Avg_rev_his
Loss

# Model selection

In [45]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [157]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import accuracy_score

from catboost import CatBoostClassifier

import mlflow

import optuna

In [167]:
categorical_cols = ['type_of_meal_plan', 'room_type_reserved', 'market_segment_type']

df = pd.get_dummies(df, columns=categorical_cols)

df_train = df.iloc[:20000]
df_test = df.iloc[20000:]

In [168]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 21771 to 33490
Data columns (total 31 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          20000 non-null  int64  
 1   no_of_children                        20000 non-null  int64  
 2   no_of_weekend_nights                  20000 non-null  int64  
 3   no_of_week_nights                     20000 non-null  int64  
 4   required_car_parking_space            20000 non-null  int64  
 5   lead_time                             20000 non-null  int64  
 6   arrival_year                          20000 non-null  int64  
 7   arrival_month                         20000 non-null  int64  
 8   arrival_date                          20000 non-null  int64  
 9   repeated_guest                        20000 non-null  int64  
 10  no_of_previous_cancellations          20000 non-null  int64  
 11  no_of_previous_b

In [169]:
len(df_test)

16275

In [170]:
X_train = df_train.drop('booking_status', axis=1)
y_train = df_train['booking_status']

X_test = df_test.drop('booking_status', axis=1)
y_test = df_test['booking_status']

In [171]:
bool_cols = X_train.select_dtypes(include='bool').columns
X_train[bool_cols] = X_train[bool_cols].astype(int)
X_test[bool_cols] = X_test[bool_cols].astype(int)

In [172]:
X_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28000 entries, 21771 to 22542
Data columns (total 30 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          28000 non-null  int64  
 1   no_of_children                        28000 non-null  int64  
 2   no_of_weekend_nights                  28000 non-null  int64  
 3   no_of_week_nights                     28000 non-null  int64  
 4   required_car_parking_space            28000 non-null  int64  
 5   lead_time                             28000 non-null  int64  
 6   arrival_year                          28000 non-null  int64  
 7   arrival_month                         28000 non-null  int64  
 8   arrival_date                          28000 non-null  int64  
 9   repeated_guest                        28000 non-null  int64  
 10  no_of_previous_cancellations          28000 non-null  int64  
 11  no_of_previous_b

In [47]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')

In [22]:
mlflow.set_experiment("regression_experiment")

2025/07/31 11:18:15 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/31 11:18:15 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/07/31 11:18:15 INFO mlflow.tracking.fluent: Experiment with name 'regression_experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/domager73/Downloads/ml/Harbour_project/mlruns/1', creation_time=1753953495465, experiment_id='1', last_update_time=1753953495465, lifecycle_stage='active', name='regression_experiment', tags={}>

In [94]:
with mlflow.start_run():
    model = LinearRegression()
    model.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    rmse = root_mean_squared_error(test_pred, y_test)
    mlflow.log_metric('rmse', rmse)
    mlflow.set_tag('workspace', 'select model')

In [100]:
with mlflow.start_run():
    model = LogisticRegression()
    model.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    train_pred = model.predict(X_train)
    accuracy_test = accuracy_score(test_pred, y_test)
    accuracy_train = accuracy_score(train_pred, y_train)
    mlflow.log_metric('accuracy_train', accuracy_train)
    mlflow.log_metric('accuracy_test', accuracy_test)
    mlflow.set_tag('workspace', 'select model')

/Users/domager73/PycharmProjects/Hoter_reser_harbour/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning:

lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [101]:
mlflow.set_experiment("decision_tree experiment")

2025/07/31 11:46:00 INFO mlflow.tracking.fluent: Experiment with name 'decision_tree experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/domager73/Downloads/ml/Harbour_project/mlruns/2', creation_time=1753955160454, experiment_id='2', last_update_time=1753955160454, lifecycle_stage='active', name='decision_tree experiment', tags={}>

In [106]:
mlflow.autolog()
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)


accuracy_test = accuracy_score(test_pred, y_test)
accuracy_train = accuracy_score(train_pred, y_train)


mlflow.log_metric('accuracy_train', accuracy_train)
mlflow.log_metric('accuracy_test', accuracy_test)
mlflow.set_tag('workspace', 'select model')

2025/07/31 11:51:59 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 1.3.0 <= scikit-learn <= 1.7.0, but the installed version is 1.7.1. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2025/07/31 11:52:02 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/07/31 11:52:02 WARNING mlflow.utils.autologging_utils: MLflow statsmodels autologging is known to be compatible with 0.14.1 <= statsmodels <= 0.14.4, but the installed version is 0.14.5. If you encounter errors during autologging, try upgrading / downgrading statsmodels to a compatible version, or try upgrading MLflow.
2025/07/31 11:52:06 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2025/07/31 11:52:06 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.7.6 <= xgboost <= 3.0.2, but the instal

In [108]:
mlflow.autolog()

mlflow.set_tag("description", "RandomForestClassifier auto log")
model = RandomForestClassifier()
model.fit(X_train, y_train)
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)


accuracy_test = accuracy_score(test_pred, y_test)
accuracy_train = accuracy_score(train_pred, y_train)


mlflow.log_metric('accuracy_train', accuracy_train)
mlflow.log_metric('accuracy_test', accuracy_test)
mlflow.set_tag('workspace', 'select model')

2025/07/31 11:55:08 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 1.3.0 <= scikit-learn <= 1.7.0, but the installed version is 1.7.1. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2025/07/31 11:55:08 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/07/31 11:55:08 WARNING mlflow.utils.autologging_utils: MLflow statsmodels autologging is known to be compatible with 0.14.1 <= statsmodels <= 0.14.4, but the installed version is 0.14.5. If you encounter errors during autologging, try upgrading / downgrading statsmodels to a compatible version, or try upgrading MLflow.
2025/07/31 11:55:08 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2025/07/31 11:55:08 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.7.6 <= xgboost <= 3.0.2, but the instal

In [111]:
mlflow.autolog()

mlflow.set_tag("description", "HistGradientBoostingClassifier auto log")
model = HistGradientBoostingClassifier()
model.fit(X_train, y_train)
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)


accuracy_test = accuracy_score(test_pred, y_test)
accuracy_train = accuracy_score(train_pred, y_train)


mlflow.log_metric('accuracy_train', accuracy_train)
mlflow.log_metric('accuracy_test', accuracy_test)
mlflow.set_tag('workspace', 'select model')

2025/07/31 11:56:36 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 1.3.0 <= scikit-learn <= 1.7.0, but the installed version is 1.7.1. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2025/07/31 11:56:36 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/07/31 11:56:36 WARNING mlflow.utils.autologging_utils: MLflow statsmodels autologging is known to be compatible with 0.14.1 <= statsmodels <= 0.14.4, but the installed version is 0.14.5. If you encounter errors during autologging, try upgrading / downgrading statsmodels to a compatible version, or try upgrading MLflow.
2025/07/31 11:56:36 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2025/07/31 11:56:36 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.7.6 <= xgboost <= 3.0.2, but the instal

In [110]:
mlflow.end_run()

In [119]:
mlflow.set_experiment("catboost_experiment")

<Experiment: artifact_location='/Users/domager73/Downloads/ml/Harbour_project/mlruns/3', creation_time=1753955944515, experiment_id='3', last_update_time=1753955944515, lifecycle_stage='active', name='catboost_experiment', tags={}>

In [120]:
mlflow.autolog()

mlflow.set_tag("description", "Catboost auto log")
model = CatBoostClassifier()
model.fit(X_train, y_train)
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

accuracy_test = accuracy_score(test_pred, y_test)
accuracy_train = accuracy_score(train_pred, y_train)

mlflow.log_metric('accuracy_train', accuracy_train)
mlflow.log_metric('accuracy_test', accuracy_test)
mlflow.set_tag('workspace', 'select model')


2025/07/31 12:01:10 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 1.3.0 <= scikit-learn <= 1.7.0, but the installed version is 1.7.1. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2025/07/31 12:01:10 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/07/31 12:01:10 WARNING mlflow.utils.autologging_utils: MLflow statsmodels autologging is known to be compatible with 0.14.1 <= statsmodels <= 0.14.4, but the installed version is 0.14.5. If you encounter errors during autologging, try upgrading / downgrading statsmodels to a compatible version, or try upgrading MLflow.
2025/07/31 12:01:10 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2025/07/31 12:01:10 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.7.6 <= xgboost <= 3.0.2, but the instal

Learning rate set to 0.042744
0:	learn: 0.6632408	total: 8.11ms	remaining: 8.11s
1:	learn: 0.6358283	total: 11.5ms	remaining: 5.74s
2:	learn: 0.6131625	total: 15.2ms	remaining: 5.04s
3:	learn: 0.5931480	total: 18.2ms	remaining: 4.53s
4:	learn: 0.5696288	total: 21.4ms	remaining: 4.26s
5:	learn: 0.5526843	total: 24.4ms	remaining: 4.03s
6:	learn: 0.5356539	total: 27.6ms	remaining: 3.91s
7:	learn: 0.5241977	total: 30.9ms	remaining: 3.83s
8:	learn: 0.5087307	total: 33.9ms	remaining: 3.73s
9:	learn: 0.4975314	total: 37.1ms	remaining: 3.68s
10:	learn: 0.4875353	total: 40.3ms	remaining: 3.62s
11:	learn: 0.4782169	total: 43.4ms	remaining: 3.57s
12:	learn: 0.4705983	total: 46.6ms	remaining: 3.54s
13:	learn: 0.4628754	total: 49.6ms	remaining: 3.49s
14:	learn: 0.4553844	total: 52.9ms	remaining: 3.47s
15:	learn: 0.4483834	total: 56ms	remaining: 3.44s
16:	learn: 0.4426202	total: 59.4ms	remaining: 3.43s
17:	learn: 0.4371452	total: 62.5ms	remaining: 3.41s
18:	learn: 0.4325236	total: 65.8ms	remaining: 

In [122]:
mlflow.end_run()

In [173]:
from sklearn.model_selection import ParameterGrid

param_grid = {
    'iterations': [2000, 3000],
    'learning_rate': [0.05, 0.1],
    'depth': [6, 10],
    'l2_leaf_reg': [3,5, 8]
}

mlflow.autolog()

for params in ParameterGrid(param_grid):
    with mlflow.start_run():
        mlflow.set_tag("description", f"CatBoost hyperparameter tuning {param_grid}")
        mlflow.set_tag("workspace", "select model")
        mlflow.set_tag("model_type", "CatBoostClassifier")
        
        mlflow.log_params(params)
        
        model = CatBoostClassifier(
            **params,
            verbose=0, 
            random_state=42
        )
        
        model.fit(X_train, y_train)
        
        test_pred = model.predict(X_test)
        train_pred = model.predict(X_train)
        
        accuracy_test = accuracy_score(y_test, test_pred)
        accuracy_train = accuracy_score(y_train, train_pred)
        
        mlflow.log_metric('accuracy_train', accuracy_train)
        mlflow.log_metric('accuracy_test', accuracy_test)
        
        mlflow.log_metric('delta_accuracy', accuracy_train - accuracy_test)
        
        print(f"Params: {params}")
        print(f"Test accuracy: {accuracy_test:.4f}")

2025/07/31 21:59:59 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 1.3.0 <= scikit-learn <= 1.7.0, but the installed version is 1.7.1. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2025/07/31 21:59:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/07/31 21:59:59 WARNING mlflow.utils.autologging_utils: MLflow statsmodels autologging is known to be compatible with 0.14.1 <= statsmodels <= 0.14.4, but the installed version is 0.14.5. If you encounter errors during autologging, try upgrading / downgrading statsmodels to a compatible version, or try upgrading MLflow.
2025/07/31 21:59:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2025/07/31 21:59:59 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.7.6 <= xgboost <= 3.0.2, but the instal

Params: {'depth': 6, 'iterations': 2000, 'l2_leaf_reg': 3, 'learning_rate': 0.05}
Test accuracy: 0.8095
Params: {'depth': 6, 'iterations': 2000, 'l2_leaf_reg': 3, 'learning_rate': 0.1}
Test accuracy: 0.8021


KeyboardInterrupt: 

In [126]:
def objective(trial):
    model = catboost.CatBoostClassifier(
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        od_wait=trial.suggest_int("od_wait", 10, 50),
        verbose=False
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

In [133]:
from optuna.samplers import TPESampler
import optuna
import catboost

sampler = TPESampler(seed=1)

In [134]:
study = optuna.create_study(study_name="catboost", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=100)

[I 2025-07-31 12:13:39,101] A new study created in memory with name: catboost
[I 2025-07-31 12:13:42,514] Trial 0 finished with value: 0.8450755287009063 and parameters: {'iterations': 475, 'learning_rate': 0.027583475549166746, 'depth': 4, 'l2_leaf_reg': 1.0551779964424746e-05, 'bootstrap_type': 'Bayesian', 'random_strength': 2.0931628460945333e-07, 'bagging_temperature': 0.923385947687978, 'od_type': 'Iter', 'od_wait': 26}. Best is trial 0 with value: 0.8450755287009063.
[I 2025-07-31 12:13:45,191] Trial 1 finished with value: 0.8561933534743202 and parameters: {'iterations': 585, 'learning_rate': 0.006892694481137703, 'depth': 8, 'l2_leaf_reg': 1.10795595820296e-06, 'bootstrap_type': 'Bayesian', 'random_strength': 0.79993910451721, 'bagging_temperature': 0.27387593197926163, 'od_type': 'IncToDec', 'od_wait': 32}. Best is trial 1 with value: 0.8561933534743202.
[I 2025-07-31 12:13:46,627] Trial 2 finished with value: 0.8322658610271904 and parameters: {'iterations': 226, 'learning_ra

In [135]:
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials:  100
Best trial:
  Value:  0.8633232628398791
  Params: 
    iterations: 162
    learning_rate: 0.020659576807258582
    depth: 8
    l2_leaf_reg: 0.8679838253640907
    bootstrap_type: Bayesian
    random_strength: 1.5305950013403581e-06
    bagging_temperature: 0.5829113245739789
    od_type: IncToDec
    od_wait: 40


In [136]:
with mlflow.start_run() as run:
    mlflow.set_tag("optimizer", "optuna")
    mlflow.set_tag("model_type", "CatBoostClassifier")
    mlflow.set_tag("description", "Optuna hyperparameter optimization")
    
    mlflow.log_metric("best_accuracy", trial.value)
    mlflow.log_params(trial.params)
    mlflow.log_param("n_trials", len(study.trials))
    
    best_model = CatBoostClassifier(**trial.params)
    best_model.fit(X_train, y_train)
    
    test_pred = best_model.predict(X_test)
    train_pred = best_model.predict(X_train)
    accuracy_test = accuracy_score(y_test, test_pred)
    accuracy_train = accuracy_score(y_train, train_pred)
    
    mlflow.log_metrics({
        "final_accuracy_test": accuracy_test,
        "final_accuracy_train": accuracy_train,
        "accuracy_gap": accuracy_train - accuracy_test
    })
    
    mlflow.catboost.log_model(best_model, "best_model")
    
    trials_df = study.trials_dataframe()
    trials_df.to_csv("optuna_trials.csv", index=False)
    mlflow.log_artifact("optuna_trials.csv")

0:	learn: 0.6720991	total: 12.1ms	remaining: 1.95s
1:	learn: 0.6525455	total: 17ms	remaining: 1.36s
2:	learn: 0.6357474	total: 23.8ms	remaining: 1.26s
3:	learn: 0.6200871	total: 35.6ms	remaining: 1.41s
4:	learn: 0.6058055	total: 43.8ms	remaining: 1.38s
5:	learn: 0.5908016	total: 60.6ms	remaining: 1.57s
6:	learn: 0.5769070	total: 72.4ms	remaining: 1.6s
7:	learn: 0.5661361	total: 86.3ms	remaining: 1.66s
8:	learn: 0.5561276	total: 108ms	remaining: 1.84s
9:	learn: 0.5457519	total: 121ms	remaining: 1.84s
10:	learn: 0.5365709	total: 135ms	remaining: 1.86s
11:	learn: 0.5277573	total: 161ms	remaining: 2.02s
12:	learn: 0.5196405	total: 174ms	remaining: 1.99s
13:	learn: 0.5115723	total: 186ms	remaining: 1.97s
14:	learn: 0.5042330	total: 197ms	remaining: 1.93s
15:	learn: 0.4972150	total: 210ms	remaining: 1.92s
16:	learn: 0.4904180	total: 222ms	remaining: 1.89s
17:	learn: 0.4835657	total: 227ms	remaining: 1.82s
18:	learn: 0.4768666	total: 236ms	remaining: 1.77s
19:	learn: 0.4701448	total: 241ms	re

2025/07/31 12:23:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


158:	learn: 0.2970819	total: 1.02s	remaining: 19.2ms
159:	learn: 0.2967035	total: 1.02s	remaining: 12.8ms
160:	learn: 0.2964366	total: 1.03s	remaining: 6.39ms
161:	learn: 0.2961713	total: 1.03s	remaining: 0us


2025/07/31 12:23:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [139]:
mlflow.catboost.log_model(best_model, "best_model")

2025/07/31 12:25:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/31 12:25:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [138]:
mlflow.end_run()